Stocks Rag

Author: Orvin Bellamy (https://github.com/orvinbellamy)

In [1]:
## Import libraries

import yfinance as yf
import pandas as pd
import openai
from openai import OpenAI
import os
import json
import fs
import time
from dotenv import load_dotenv
import logging as log
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from filehandler import FileHandler
from yfinancehandler import YFHandler
from eventhandler import EventHandler, ThreadManager
from agenthandler import AgentHandler
from stockanalyzer import analyze_stock, stock_data_setup
import importlib

In [2]:
## configruation

FILE_PATH = 'openai_upload_files/'
OPENAI_DIC_FILE_NAME = 'openai_files.json'

# Load schemas from JSON file
with open('config/dataframe_schemas.json', 'r') as f:
    schemas = json.load(f)

# Initialize an empty list to store the stocks
stocks_list = []

# Open dic_files
with open(f'{FILE_PATH}{OPENAI_DIC_FILE_NAME}', 'r') as f:
    dic_files = json.load(f)

with open('config/assistants.json', 'r') as json_file:
    dic_assistants= json.load(json_file)

### Stocks Data

In [ ]:
analysis = analyze_stock(
	ticker= ['MSFT'], 
	dic_files=dic_files,
	dic_assistants=dic_assistants
)

In [5]:
print(analysis)

NameError: name 'analysis' is not defined

In [3]:
ticker = ['MRNA']

client = OpenAI()

file_stocks = stock_data_setup(client=client, ticker=ticker, type='price', dic_files=dic_files)
file_cashflow = stock_data_setup(client=client, ticker=ticker, type='cash', dic_files=dic_files)
file_income_stmt = stock_data_setup(client=client, ticker=ticker, type='income', dic_files=dic_files)

# Have to manually update the tool_resources because the file_id can change
dic_assistants['fin_analyst']['tool_resources'] = {
	'code_interpreter': {'file_ids': [dic_files['df_stocks.csv']]}
}

with open('config/assistants.json', 'w') as json_file:
	json.dump(dic_assistants, json_file)
	print(f'assistants.json file has been updated')

fin_analyst = AgentHandler(
	client = client, 
	new=False,
	assistant_name = 'fin_analyst',
	dic_file = dic_assistants,
	dic_file_name = 'assistants.json',
	dic_file_path='config/'
	)

fin_consultant = AgentHandler(
	client = client,
	new=False,
	assistant_name = 'fin_consultant',
	dic_file = dic_assistants,
	dic_file_name = 'assistants.json',
	dic_file_path='config/'
	)

d:\Documents\Python Repo\stock_rag\yfinancehandler.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)


file name: df_stocks.csv, file id: file-C9RWbrhaJ259ZHBBLVYxCoSO has been deleted
file name: df_stocks.csv is uploaded, new file id: file-X0GLBQU5z3XD15lJt5pMmSYU
openai_upload_files/openai_files.json file has been updated


d:\Documents\Python Repo\stock_rag\yfinancehandler.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)


file name: df_cashflow.csv, file id: file-ZZBv1g7qkZmJpacsB0EbJzd6 has been deleted
file name: df_cashflow.csv is uploaded, new file id: file-JWq6ARhok9tFLaYssmR4HwHx
openai_upload_files/openai_files.json file has been updated


d:\Documents\Python Repo\stock_rag\yfinancehandler.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)


file name: df_income_stmt.csv, file id: file-HO5yZjDt6WACDz8oEl9ZmkZb has been deleted
file name: df_income_stmt.csv is uploaded, new file id: file-1V2UjkzdkfnnhzebK0M4ZYPM
openai_upload_files/openai_files.json file has been updated
assistants.json file has been updated
Assistant has been updated, name: fin_analyst, id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
Assistant has been updated, name: fin_consultant, id: asst_mPlBQ4ZF8YIF238DWqWRhV4U


In [21]:
prompt_start = f"This is the financial consultant.\
	The client wants advice on the following stocks.\
	I need you to provide me your analysis on these stocks so I can provide the appropriate recommendations.\
	You will be provided with the data of these stocks in .csv format. Use them as you see fit.\
	Stocks: {ticker}"

thread = ThreadManager(
	client=client,
	prompt=prompt_start
)

thread.run_thread(
	assistant=fin_analyst
	# prompt=prompt_start,
	# attachments=[file_stocks.file_id, file_cashflow.file_id, file_income_stmt.file_id]
)

next_prompt = thread.last_message

thread.run_thread(
	assistant=fin_consultant,
	prompt=next_prompt
)


assistant > code_interpreter

import pandas as pd

# Load the CSV file to check the structure of the data
file_path = '/mnt/data/file-X0GLBQU5z3XD15lJt5pMmSYU'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head()
assistant > Here are the information gathered from the financial analyst: 

These are the stocks the client wants advice on: 
1. MRNA

### Analysis of Moderna Inc. (MRNA)

**Historical Price Data (Summary)**:
- The dataset includes historical prices from December 7, 2018.
- Key columns include: Date, Open, High, Low, Close, Volume, Dividends, and Stock Splits. 

**Key Observations**:
1. **Price Movements**:
   - **Open**: The stock opened at $22 on December 7, 2018, and experienced a significant range in the price movements subsequently.
   - **Close**: The closing price varied considerably from the inception, starting at $18.6 on the first trading day.
   
2. **Volume Trends**:
   - High trading volumes indicate s

In [22]:
next_prompt = 'This message is for testing. Can you give a short sample response.'

thread.run_thread(
	assistant=fin_consultant,
	prompt=next_prompt
)

print(thread.last_message)


assistant > ### Recommendation for Moderna Inc. (MRNA)

**Overview**:
- Moderna is a leading biotechnology company known for its mRNA technology, notably its COVID-19 vaccine.
- The stock has shown significant price volatility, which is common for biotech firms.

**Strengths**:
- **Innovative Leader**: Strong R&D pipeline with potential for future growth.
- **Market Presence**: Established brand and substantial market share from COVID-19 vaccine.

**Weaknesses**:
- **Volatility**: High price fluctuations influenced by news and product announcements.
- **No Dividends**: May not appeal to those seeking regular income.

**Fit with Investment Strategy**:
- **High-Risk Tolerant**: Suitable for growth-oriented investors looking for high potential returns.
- **Conservative Investors**: May not be ideal due to volatility and the absence of dividend payouts.

**Recommendation**:
- **Growth-Oriented**: Allocate a portion of funds to MRNA, keeping a close watch on market trends and news.
- **Bal

In [23]:
thread.df_messages
# client.beta.threads.messages.list(thread_id=thread.thread_id).data
# messages_data[0]
# type(file_cashflow)

,message_id,assistant_id,created_at,file_ids,role,run_id,message_text,_msg_loc
0,msg_jNKhekiTCZtm8w4FPvX15gAj,None,1726556528,[],user,None,This is the financial consultant.\tThe client ...,0
1,msg_ZnRQdMDKHoD1H5rtlXqHchrD,asst_Mqf9cO1sDTOd4UMYwcFfIQrA,1726556533,[],assistant,run_SUXZgjKFGQXWadJINBkRKVKT,Here are the information gathered from the fin...,1
2,msg_Uk5Nu4x58DtDkKMAq2hD1caL,None,1726556545,[],user,None,Here are the information gathered from the fin...,2
3,msg_OeTIVYGnagoJeGndj95UJJFG,asst_mPlBQ4ZF8YIF238DWqWRhV4U,1726556546,[],assistant,run_e3Wu9olqlU7Ts7zIrWRpeAoY,### Recommendation for Moderna Inc. (MRNA)\n\n...,3
4,msg_2oqodrxs428y1uKMA8I38ikI,None,1726556574,[],user,None,This message is for testing. Can you give a sh...,4
5,msg_tXf7w2DLQf3jXx2Vihici8De,asst_mPlBQ4ZF8YIF238DWqWRhV4U,1726556575,[],assistant,run_zhpjoi5bMscVc2i6fZWGL35t,### Recommendation for Moderna Inc. (MRNA)\n\n...,5


In [31]:
thread.df_messages

,message_id,assistant_id,created_at,file_ids,role,run_id,message_text,_msg_loc
0,msg_jNKhekiTCZtm8w4FPvX15gAj,None,1726556528,[],user,None,This is the financial consultant.\tThe client ...,0
1,msg_ZnRQdMDKHoD1H5rtlXqHchrD,asst_Mqf9cO1sDTOd4UMYwcFfIQrA,1726556533,[],assistant,run_SUXZgjKFGQXWadJINBkRKVKT,Here are the information gathered from the fin...,1
2,msg_Uk5Nu4x58DtDkKMAq2hD1caL,None,1726556545,[],user,None,Here are the information gathered from the fin...,2
3,msg_OeTIVYGnagoJeGndj95UJJFG,asst_mPlBQ4ZF8YIF238DWqWRhV4U,1726556546,[],assistant,run_e3Wu9olqlU7Ts7zIrWRpeAoY,### Recommendation for Moderna Inc. (MRNA)\n\n...,3
4,msg_2oqodrxs428y1uKMA8I38ikI,None,1726556574,[],user,None,This message is for testing. Can you give a sh...,4
5,msg_tXf7w2DLQf3jXx2Vihici8De,asst_mPlBQ4ZF8YIF238DWqWRhV4U,1726556575,[],assistant,run_zhpjoi5bMscVc2i6fZWGL35t,### Recommendation for Moderna Inc. (MRNA)\n\n...,5


In [30]:
# existing_message_id = [dic['message_id'] for dic in thread.messages]

thread.df_messages = thread.df_messages.loc[0:1,:]

thread.get_last_message()
# thread.messages[-1]

get_last_message initiated
asst_id = blank
message.assistant_id != asst_id, record previous message first
message.assistant_id != asst_id, record previous message first
message.assistant_id != asst_id, record previous message first


In [20]:
thread.delete_thread()

thread: thread_mPIjTp06Kua0Y9HZvV0eUdh3 has been deleted.
